In [221]:
import matplotlib.pyplot as plt
from numba import njit, vectorize, prange, guvectorize, float64
import numpy as np
import math

In [222]:
@njit(nogil=True, fastmath=True)
def kurtosis(xi4, xi3, xi2, mean_value, size):
    return ((xi4 - 4 * mean_value * xi3 + 6 * np.power(mean_value, 2) * xi2 - 3 * size * np.power(mean_value,
                                                                                                  4)) * size / np.power(
        xi2 - size * np.power(mean_value, 2), 2))

In [223]:
def kurtosis2(xi4, xi3, xi2, mean_value, size):
    return ((xi4 - 4 * mean_value * xi3 + 6 * np.power(mean_value, 2) * xi2 - 3 * size * np.power(mean_value,
                                                                                                  4)) * size / np.power(
        xi2 - size * np.power(mean_value, 2), 2))

In [224]:
@guvectorize([(float64, float64, float64, float64, float64, float64)], '(),(),(),(),()->()')
def kurtosis3(xi4, xi3, xi2, mean_value, size, res):
    res = ((xi4 - 4 * mean_value * xi3 + 6 * np.power(mean_value, 2) * xi2 - 3 * size * np.power(mean_value,
                                                                                                  4)) * size / np.power(
        xi2 - size * np.power(mean_value, 2), 2))


a = np.random.randn(10000)
@guvectorize([(float64[:], float64)], '(n)->()')
def rms(data, res):
    res = np.sqrt(np.power(data, 2).sum() / len(data))
@njit(nogil=True, fastmath=True)
def twodtd_rms(data, size):
    xi2 = np.sum(np.power(data, 2))
    rms_value = np.sqrt(xi2 / size)
    return rms_value, xi2
def twodtd_rms2(data, size):
    xi2 = np.sum(np.power(data, 2))
    rms_value = np.sqrt(xi2 / size)
    return rms_value, xi2
@njit(nogil=True, fastmath=True)
def twodtd_crest(data, size):
    max_value = np.max(np.abs(data))  # 由max(data)改为max(abs(data))
    xi2 = np.sum(np.power(data, 2))
    crest = max_value / np.sqrt(xi2 / size)
    return crest, max_value, xi2
def twodtd_crest2(data, size):
    max_value = np.max(np.abs(data))  # 由max(data)改为max(abs(data))
    xi2 = np.sum(np.power(data, 2))
    crest = max_value / np.sqrt(xi2 / size)
    return crest, max_value, xi2
@njit(nogil=True, fastmath=True)
def twodtd_kurtosis(data, size):
    mean_value = np.mean(data)
    xi4 = np.sum(np.power(data, 4))
    xi3 = np.sum(np.power(data, 3))
    xi2 = np.sum(np.power(data, 2))
    kur = kurtosis(xi4, xi3, xi2, mean_value, size)
    return kur, mean_value, xi4, xi3, xi2
def twodtd_kurtosis2(data, size):
    mean_value = np.mean(data)
    xi4 = np.sum(np.power(data, 4))
    xi3 = np.sum(np.power(data, 3))
    xi2 = np.sum(np.power(data, 2))
    kur = kurtosis(xi4, xi3, xi2, mean_value, size)
    return kur, mean_value, xi4, xi3, xi2

In [225]:
%timeit twodtd_kurtosis(a,100)

17.8 µs ± 6.97 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [226]:
%timeit twodtd_kurtosis2(a,100)

614 µs ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [227]:
print(twodtd_kurtosis(a, 100)[:10],twodtd_kurtosis2(a, 100)[:10])


(0.02958161293681379, -0.00945942326049086, 29724.953803688077, -716.580686222863, 10020.54235384299) (0.029581612936813807, -0.009459423260490854, 29724.953803688062, -716.5806862228638, 10020.542353842984)
